In [1]:
from osgeo_utils import gdal_retile
from osgeo import gdal
from osgeo import ogr
import os

from PIL import Image


In [8]:
# Retiles source images to a given size.
def retile_img(source, size, target_dir):
    os.mkdir(target_dir)
    os.mkdir(target_dir+'/tif')
    os.mkdir(target_dir+'/jpg')

    target_jpg = target_dir + '/jpg'
    target_tif = target_dir + '/tif'
    
    script_jpg = f'python /Users/michaelurson/miniconda3/pkgs/gdal-3.6.3-py310h0b34360_1/bin/gdal_retile.py -ps {size} {size} -targetDir {target_jpg} -of JPEG -co "QUALITY=95" {source}'
    
    script_tif = f'python /Users/michaelurson/miniconda3/pkgs/gdal-3.6.3-py310h0b34360_1/bin/gdal_retile.py -ps {size} {size} -targetDir {target_tif} -of GTiff {source}'
    
    print('creating jpg')
    os.system(script_jpg)
    print('jpg done')
    print('creating tif')
    os.system(script_tif)
    print('tif done')

In [9]:
test_img = './spring_vs_autumn/20180419_005.tif'
retile_img(test_img, 500, './spring_vs_autumn/spring_2018_new')

creating jpg
0...10...20...30...40...50...60...70...80...90...100 - done.
jpg done
creating tif
0...10...20...30...40...50...60...70...80...90...100 - done.
tif done


In [1]:
# Remove images that are only white.
def remove_whites(filename):
    a = Image.open(filename).convert('L').getcolors()
    if len(a) == 1:
        os.remove(filename)

In [ ]:
# Retiles image and then removes tiles that are only white.

test_img = './spring_vs_autumn/20220302_005.tif'
new_path = './spring_vs_autumn/to_predict/spring_2022_new'
retile_img(test_img, 500, new_path)

p = './spring_vs_autumn/to_predict/spring_2022_new/jpg/'

jpg_list = [x for x in os.listdir(p) if x[-3:] == 'jpg']

[remove_whites(p+x) for x in jpg_list]

[os.remove(p+x) for x in os.listdir(p) if x[-3:]!='jpg']

In [ ]:
# Selection of tifs based on JPG selection:
# This is useful if a sample has been selected based on one file type, and then the corresponding files having a different type are required.
import shutil

def copy_files(desired_extension, source_orig, source_desired):

    if desired_extension.lower() == 'jpg':
        ext_new = 'jpg'
        ext_existing = 'tif'
    else:
        ext_new = 'tif'
        ext_existing = 'jpg'
    
    # Create new directory
    try:
        os.mkdir(source_orig+ext_new)
    except FileExistsError:
        print('folder exists')
        pass

    # Create list of original files
    org_files = []
    for file in os.listdir(source_orig):
        if file[-3:] == ext_existing:
            org_files.append(file)
    
    for file in org_files:
        print(source_desired+file[0:-3]+ext_new, source_orig+ext_new)
        try:
            shutil.copy(source_desired+file[0:-3]+ext_new, source_orig+ext_new)
        except FileNotFoundError:
            print(f'file {file} not found')
            pass

In [ ]:
copy_files('jpg', './spring_vs_autumn/filtered', './spring_vs_autumn/autumn_2018_tiles')

In [ ]:
# This thing takes files from one directory and shuffles the name around to put the period at the end.
# Useful for annotation in DashDoodler.
import copy

working_dir = '/Users/michaelurson/Desktop/Doodler/spring_vs_autumn/testing_datasets/combined/'
new_names = []
files = os.listdir(working_dir)
for file in files:
    source_name = file
    components = file.split('_')
    img_id = '_'.join(components[1:])
    img_id = img_id.replace('.jpg','_'+components[0]+'.jpg')
    new_names.append(img_id)
    try:
        os.rename(working_dir+source_name,working_dir+img_id)
    except:
        IsADirectoryError
        pass
new_names


In [ ]:
# This thing copies the equivalent photos for a given season to another directory.
import os
import shutil

output_dir = './spring_vs_autumn/testing_datasets/autumn_2018/'

source_dir = './spring_vs_autumn/testing_datasets/spring_2018/'
desired_dir = './spring_vs_autumn/autumn_2018_tiles/jpg/'

src_list = os.listdir(source_dir)
desired_list = os.listdir(desired_dir)

src_date = src_list[0][0:8]
desired_date = desired_list[0][0:8]

src_wanted = []
for file in src_list:
    desired_name = '_'.join(file.split('_')[1:])
    
    desired_name=desired_date+'_'+desired_name
    src_wanted.append(desired_name)
    print(desired_name)

for file in src_wanted:
    try:
        shutil.copy(desired_dir+file, output_dir+file)
    except FileNotFoundError:
        print('file not found')

In [1]:
# Rename to original names
import os

source_dir = './spring_vs_autumn/sampled/autumn/'
src_list = os.listdir(source_dir)
for file in src_list:
    # Split into components
    components = file.split('_')
    # Reshuffle name
    new_name = components[-1][0:-4] + "_" + "_".join(components[0:3]) +".jpg"
    os.rename(source_dir+file, source_dir+new_name)
    


In [ ]:
# Randomly sample 100 images that are not present in the 'done' directory.
import random
import shutil
import os

copy_to = './spring_vs_autumn/spring_2018_tiles/avail_to_sample/'
source_dir = './spring_vs_autumn/spring_2018_tiles/jpg/'
done_dir = './spring_vs_autumn/annotated/spring/usable/images/'

done_samples = os.listdir(done_dir)
jpgs = os.listdir(source_dir)

# Only return samples that haven't been annotated
remaining = list(set(jpgs).difference(set(done_samples)))

num_samples = 50
avail = []
for i in range(0,num_samples + 1):
    avail.append(jpgs.pop(random.randrange(len(jpgs))))
    #tino_samples.append(jpgs.pop(random.randrange(len(jpgs))))

for file in avail:
    shutil.copyfile(source_dir+file, copy_to+file)

#for file in tino_samples:
#    shutil.copyfile(source_dir+file, tino_dir+file)